# Task 1: Principal Component Analysis

This notebook is a showcase of our `pca` module, which implements Principal Component Analysis (PCA).

PCA is a method for dimensionality reduction, performing well on affine-linear data.

First, the data is centered by subtracting the average datapoint from each datapoint. Then, we perform singular value decomposition on the centered data.

The trick is to reverse the singular value decomposition by setting the least important singular values to 0, and then multiplying the matrices back together. This is possible because the singular values are ordered by magnitude, and the least important singular values are the ones that contribute the least to the variance in the data.

In [ ]:
import init_notebook
from test_module import test_function
from pca import PCA
from matplotlib import pyplot as plt

# Helper module for visualization in all notebooks
import notebook_utils as utils

import numpy as np
import scipy as sc

test_function()

# Principal Component Analysis
We load the data from a file and perform PCA.

It's apparent that one of the principal components accounts for over 99.3% of the variance in the data, which motivates us to the assumption that the other principal component is just noise.

In [ ]:
with open("pca_dataset.txt", 'r') as file:
    # Load data inta a Python ndarray, shape (100, 2)
    global data_matrix
    data_matrix = np.loadtxt(file, delimiter=' ')
    assert data_matrix.shape == (100, 2)

pca_result = PCA.pca(data_matrix)

# This is how we can access the data of our PCA result
U, S, Vh, mean = pca_result
E = pca_result.energy

print(f'Energies: {E}\nSingular values ordered by magnitude: {S}')

# Reversing PSA to verify correctness

In [ ]:
data_matrix_reconstructed = pca_result.reverse_pca(2)

# Before and after display side-by-side
display_side_by_side = False
if display_side_by_side:
    for o, r in zip(data_matrix, data_matrix_reconstructed):
        print(f'o: {o}, r: {r}')

assert np.allclose(data_matrix, data_matrix_reconstructed)

# Plotting data

The plot looks suspiciously linear, which further supports the assumption that a linear model is very suitable for this dataset. The green principal component seems to be just noise, with no discernable patterns. The points align strongly with the red principal component.

PSA is a great approach for data that shows affine-linear behavior, as opposed to data on curved manifolds. This is why it makes sense to use PSA for this data.

In [ ]:
utils.plot_data_with_pcs(data_matrix, Vh)

# Approximate 1D

Eliminating the lesser principal component and approximating the data with only the first principal component, the data is approximated to a 1D line. This is done by simply setting the second singular value to 0 inside the matrix of singular values given by `S`, and then reversing the PCA by multiplying the matrices `U`, `S`, and `Vh`.

In [ ]:
# Supply 1 to reverse_pca to approximate the data to 1D
approximated_data = pca_result.reverse_pca(1)

utils.plot_data_with_pcs(approximated_data, pca_result.Vh)

# Image

We load the racoon image in gray and perform PCA on it.

**Note:** `scipy.misc.face` from the exercise sheet is deprecated because `scipy.misc` is deprecated. We use `scipy.datasets.face` instead.

Scaling the image to 249x185 pixels, we know that the image has **185 principal components.**

In [ ]:
# Load image as `ndarray`
my_image = sc.datasets.face(gray=True)
my_image = utils.rescale_greyscale_img(my_image, 249, 185)

plt.imshow(my_image, cmap='gray')

# PCA on image
We perform PCA on the image, and print some of the singular values.

In [ ]:
# We use the columns as datapoints, hence specify the flag. This 'flattens' the image
# The flag also ensures that the reconstructed image is transposed back to the original shape
pca_result_img = PCA.pca(my_image, treat_columns_as_datapoints=True)

utils.print_pca_info(pca_result_img, 5)

# Visualization of reconstructions

We visualize for different numbers of components.

Slight quality detriments are noticeable starting from 50 components, especially around the racoon's fur.

The image with 10 components is barely recogniziable despite preserving over 83% of the energy. The racoon's face is still possible to identify.

In [ ]:
# Visualize reconstructions with different numbers of principal components
for num_components in [pca_result_img.S.shape[0], 120, 50, 10]:
    utils.plot_reconstructed_image(pca_result_img, num_components, my_image.shape)


# Energy loss
To lose less than 1% of the energy, we need to preserve 76 principal components.

In [ ]:
# Find the number of components needed to retain a percentage of the energy
energy_threshold = 0.99
n = pca_result_img.min_components_until(energy_threshold)

print(f'Number of components needed to retain 99% of the energy: {n}')
print(f'We can remove {pca_result_img.S.shape[0] - n} dimensions from our image\'s columns while retaining {energy_threshold*100}% of the information')

# Part 3: Vadere Trajectory Analysis

Given a dataset of 15 pedestrians with 1000 2D coordinates each, we load the data to perform PCA on it.

First, we visualize the data for the **first two pedestrians:**

In [ ]:
with open("data_DMAP_PCA_vadere.txt", 'r') as file:
    # Load data inta a Python ndarray, shape (100, 2)
    global trajectory_matrix
    trajectory_matrix = np.loadtxt(file, delimiter=' ')
    assert trajectory_matrix.shape == (1000, 30)


Part 3.1: Visualizing the path of the ﬁrst two pedestrians in the two-dimensional space

In [ ]:
utils.plot_pedestrian_figure_variant1(trajectory_matrix)
utils.plot_pedestrian_figure_variant2(trajectory_matrix)

# Observations

- Each trajectory starts and ends at the starting point
- The pedestrians appear to be walking in circles
- There seems to be a lot of overlap
- The pedestrians are moving in thin rings. Perhaps PCA can thin out the rings while preserving the overall shape of the trajectories, leading to a high-energy preservation of the data even with a low number of principal components.
- This is a hypothesis that we can test out

Part 3.1: Visualizing the path of the ﬁrst two pedestrians in the two-dimensional space: Gradient Coloring

# The data represents 15 pedestrians with coordinates (x, y). The columns are the timesteps

We visualize the points for a given timestep

In [ ]:
pca_result = PCA.pca(trajectory_matrix)

utils.print_pca_info(pca_result, 3)

utils.reconstruct_and_plot_trajectory(pca_result, 2)
utils.reconstruct_and_plot_trajectory(pca_result, 3)

# Observations

- Two principal components accurately reconstruct the vague shape of the trajectory.
- However, drawing the velocity arrows from the original data, it is apparent that the velocity arrows deviate slightly from the original trajectory.
- Three principal components are much better at imitating the original shape of the trajectory.

Overall, it is impressive that the PCA is able to reconstruct the seemingly non-linear trajectory very accurately with only 3 principal components, given that it is a 90% reduction in dimensionality. We hypothesize that this is because the trajectories have many points that are very close to each other, allowing the PCA to act as a locally linear approximator. Even the rounded corners are well approximated by the PCA, which can be explained by the fact that they contain lots of data points. 

We might want to try PCA with less than 1000 data points, to see if the PCA is able to reconstruct the trajectory with less data points. The data would still have 30 principal components, but the PCA would have less data points to work with, possibly allowing for a more equal distribution of the energies. This would be a good test of the PCA's ability to approximate the trajectory with less data points.